In [41]:
import selenium
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import WebDriverException, TimeoutException
from tqdm import tqdm
import time

In [43]:

class Scraper:
#to access website
    def __init__(self):
        self.driver = webdriver.Chrome(ChromeDriverManager().install())
        url = "https://www.zoopla.co.uk/new-homes/property/london/?q=London&results_sort=newest_listings&search_source=new-homes&page_size=25&pn=1&view_type=list"
        self.driver.get(url)
        time.sleep(2)
        self.delay = 10

    def load_and_accept_cookies(self):
        try:
            WebDriverWait(self.driver, self.delay).until(EC.presence_of_element_located((By.XPATH, '//*[@id="gdpr-consent-notice"]')))
            # print("Frame Ready!")
            self.driver.switch_to.frame('gdpr-consent-notice')
            accept_cookies_button = WebDriverWait(self.driver, self.delay).until(EC.presence_of_element_located((By.XPATH, '//*[@id="save"]')))
            # print("Accept Cookies Button Ready!")
            accept_cookies_button.click()
            time.sleep(1)

        except TimeoutException:
            print("Loading took too much time!")

        return self.driver
        
    def get_links(self):
        
        # self.load_and_accept_cookies()
        links = []
        time.sleep(5)
        link_list = self.driver.find_element(By.XPATH, '//div[@class="css-1itfubx er561h10"]')
        list = link_list.find_elements(By.XPATH, './div')
        for sites in list:
            link_tag = sites.find_element(By.TAG_NAME, 'a')
            links.append(link_tag.get_property('href'))
        print(f"There are {len(links)} properties on this page")
        return links
        

    def next_page(self):
        
        big_list = []
        delay = 10
        for i in range(5):
            
            big_list.extend(self.get_links())
            time.sleep(10)
            page = self.driver.find_element(By.XPATH, '//li[@class="css-qhg1xn-PaginationItemPreviousAndNext-PaginationItemNext eaoxhri2"]')
            page.click()
            try:
                WebDriverWait(self.driver, 3).until(EC.presence_of_element_located((By.XPATH, '//div[@class="css-1vc84cj-CancelButtonWrapper e13xjwxo5"]/button'))).click()
                print("Frame Ready!")
            except:
                pass
        print(f"There are {len(big_list)} properties on this pages")

   
       

sc = Scraper()
load = sc.load_and_accept_cookies()
# sc.get_links()
sc.next_page()



[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 103.0.5060
[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome
[WDM] - Driver [/Users/pearl/.wdm/drivers/chromedriver/mac64_m1/103.0.5060.53/chromedriver] found in cache
/var/folders/k4/lsf7875j0pd9cxljb33jtqb40000gn/T/ipykernel_22898/3170854622.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  self.driver = webdriver.Chrome(ChromeDriverManager().install())


There are 25 properties on this page
Frame Ready!
There are 25 properties on this page
There are 25 properties on this page
There are 25 properties on this page
There are 25 properties on this page
There are 125 properties on this pages
